### Intsanciation of ChatGroq

In [32]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()
llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    groq_api_key=os.getenv('API_KEY')
)
response=llm.invoke("What is Data Mining")
print(response.content)

Data mining is the process of discovering patterns, trends, and insights from large datasets using statistical, machine learning, and other analytical techniques. The goal of data mining is to extract valuable information from data and turn it into understandable structures for further use in decision-making, prediction, and other business or research objectives.

Data mining involves several steps, including data cleaning, data integration, data selection, data transformation, data mining, pattern evaluation, and knowledge presentation. Data mining techniques include clustering, classification, association rule learning, anomaly detection, and regression analysis.

Data mining has applications in various fields, such as marketing, finance, healthcare, manufacturing, and telecommunications. It can help organizations to improve their decision-making, optimize their operations, and gain a competitive advantage by leveraging the insights gained from their data.

However, data mining also 

### Setup the chromadb vector database

In [33]:
import chromadb
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="my_collection")


UniqueConstraintError: Collection my_collection already exists

In [10]:
collection.add(
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    ids=["id1", "id2"]
)
#here we can also add a metadata and it is useful to khow the sources form where it comes the documents


C:\Users\allak\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [01:18<00:00, 1.06MiB/s]


In [11]:
all_docs=collection.get()
all_docs

{'ids': ['id1', 'id2'],
 'embeddings': None,
 'metadatas': [None, None],
 'documents': ['This is a document about pineapple',
  'This is a document about oranges'],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [12]:
results=collection.query(
    query_texts=["Query is about fruit"],
    n_results=2
)
results

{'ids': [['id1', 'id2']],
 'distances': [[0.9236728549003601, 1.0640615224838257]],
 'metadatas': [[None, None]],
 'embeddings': None,
 'documents': [['This is a document about pineapple',
   'This is a document about oranges']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [13]:
collection.delete(ids=all_docs['ids'])
collection.get()


{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [ ]:
https://careers.bcg.com/us/en/job/BCG1US25997EXTERNALENUS/Data-Engineering-Intern-X-Delivery?utm_source=linkedin&utm_medium=phenom-feeds

### Email generator
Web Base Loader from Langchain will do a web scaring from a giving url 


In [23]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://eeho.fa.us2.oraclecloud.com/hcmUI/CandidateExperience/en/sites/jobsearch/job/256481?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic")
page_data=loader.load().pop().page_content
print(page_data)













Oracle




























### Prompt template for our llm:

In [37]:
from langchain_core.prompts import PromptTemplate

# Instantiation using from_template (recommended)
prompt_extarct= PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scaped text is from the caree's page of a website
    Your job is to extract the job postings and return them in JSON format containing the following keys:
    `role`,`experience`,`skills` and `description`.
    Only return the valid JSON.
    ### VALID JSON(NO PREAMBLE):

    """
)
chain_extract=prompt_extarct | llm
response=chain_extract.invoke(input={'page_data':page_data})
print(response.content)


[
  {
    "role": "Cloud Software Engineer",
    "experience": "3-5 years",
    "skills": [
      "Java",
      "Kubernetes",
      "Docker"
    ],
    "description": "Seeking a skilled Cloud Software Engineer with 3-5 years of experience in Java, Kubernetes, and Docker. The ideal candidate will have a strong background in cloud computing and a passion for staying up-to-date with the latest technologies. Responsibilities include designing, developing, and maintaining cloud-based applications, as well as collaborating with cross-functional teams to ensure the successful deployment of our products."
  },
  {
    "role": "Data Analyst",
    "experience": "1-3 years",
    "skills": [
      "SQL",
      "Python",
      "Data Visualization"
    ],
    "description": "We are looking for a detail-oriented Data Analyst with 1-3 years of experience in SQL, Python, and Data Visualization. The successful candidate will have a strong understanding of data analysis and reporting, and will be respons

### Convert the output to a JSON format 

In [38]:
from langchain_core.output_parsers import JsonOutputParser
json_parser= JsonOutputParser()
json_res=json_parser.parse(response.content)
json_res

[{'role': 'Cloud Software Engineer',
  'experience': '3-5 years',
  'skills': ['Java', 'Kubernetes', 'Docker'],
  'description': 'Seeking a skilled Cloud Software Engineer with 3-5 years of experience in Java, Kubernetes, and Docker. The ideal candidate will have a strong background in cloud computing and a passion for staying up-to-date with the latest technologies. Responsibilities include designing, developing, and maintaining cloud-based applications, as well as collaborating with cross-functional teams to ensure the successful deployment of our products.'},
 {'role': 'Data Analyst',
  'experience': '1-3 years',
  'skills': ['SQL', 'Python', 'Data Visualization'],
  'description': 'We are looking for a detail-oriented Data Analyst with 1-3 years of experience in SQL, Python, and Data Visualization. The successful candidate will have a strong understanding of data analysis and reporting, and will be responsible for extracting, cleaning, and interpreting data to help drive business d

In [39]:
type(json_res)

list

In [42]:
import pandas as pd
df=pd.read_csv("my_portfolio.csv")


### Inserting the records into Chromadb 
client --->create a chromadb in memory 
persistentclient --->create a chromadb on disk 

In [51]:
import chromadb
import uuid

client=chromadb.PersistentClient('vectorestore')
collection=client.get_or_create_collection(name='portfolio')
if not collection.count():
    for _,row in df.iterrows():
        collection.add(documents=["Techstack"],
                       metadatas={"links":row["Links"]},
                       ids=[str(uuid.uuid4())])

In [60]:
job=json_res[0]
job['skills']

['Java', 'Kubernetes', 'Docker']

In [62]:

links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])

links

[[{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}]]

### Another prompt template for generating email

In [63]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Experienced Cloud Software Engineer with Java, Kubernetes, and Docker Expertise - AtliQ Can Help

Dear Hiring Manager,

I hope this email finds you well. I came across your job posting for a Cloud Software Engineer with 3-5 years of experience in Java, Kubernetes, and Docker. As a Business Development Executive at AtliQ, I would like to bring to your attention our expertise in fulfilling your needs for this role.

At AtliQ, we have a proven track record of designing, developing, and maintaining cloud-based applications using Java, Kubernetes, and Docker. Our team of skilled engineers has successfully delivered numerous projects, ensuring seamless integration, scalability, and cost reduction for our clients. I am confident that our experience and capabilities make us an ideal partner for your company.

To showcase our portfolio, I would like to highlight our work with React Native and Magento, which you can find here:
https://example.com/react-native-portfolio
https://example.c